In [5]:
# 必要なモジュールのインポート
import boto3  # AWSのサービスを利用するときに必須
import json  # json形式のデータを処理する
from PIL import Image, ImageDraw  # pillow の Image モジュール 画像処理に使用する

In [15]:
def create_collection():
    # コレクションIDを入力
    collection_id = input("コレクションIDを入力: ")
    # サービスクライアントを作成
    rekognition = boto3.client('rekognition')

    # コレクションを作成する
    response = rekognition.create_collection(CollectionId=collection_id)

    # 結果のjsonを整形して表示
    print(json.dumps(response, indent=4))

In [16]:
def add_image_to_collection():
    # コレクションIDと画像ファイル名を入力
    collection_id = input("コレクションIDを入力: ")
    image_file = input("登録する画像ファイル名を入力: ")
    # サービスクライアントを作成
    rekognition = boto3.client('rekognition')

    # 画像を読み込む
    with open(image_file, 'rb') as image:
        response = rekognition.index_faces(
            CollectionId=collection_id,
            Image={'Bytes': image.read()},
            ExternalImageId=image_file,
            DetectionAttributes=['ALL']
        )

    # 結果のjsonを整形して表示
    print(json.dumps(response, indent=4))
    # 登録された顔の数を表示
    print(f"登録された顔の数: {len(response['FaceRecords'])}")

In [27]:
def search_face():
    # コレクションIDと画像ファイル名を入力
    collection_id = input("コレクションIDを入力: ")
    image_file = input("検索する画像ファイル名を入力: ")
    # サービスクライアントを作成
    rekognition = boto3.client('rekognition')

    # 画像を読み込む
    with open(image_file, 'rb') as image:
        response = rekognition.search_faces_by_image(
            CollectionId=collection_id,
            Image={'Bytes': image.read()},
            MaxFaces=1,
            FaceMatchThreshold=80
        )

    # 結果のjsonを整形して表示
    print(json.dumps(response, indent=4))

    # 顔が見つかった場合
    if response['FaceMatches']:
        # 画像を開く
        image = Image.open(image_file)
        width, height = image.size
        # 顔の位置情報を取得
        face = response['FaceMatches'][0]['Face']['BoundingBox']
        left = int(face['Left'] * width)
        top = int(face['Top'] * height)
        right = left + int(face['Width'] * width)
        bottom = top + int(face['Height'] * height)
        # 顔部分を切り取る
        face_image = image.crop((left, top, right, bottom))
        # 顔部分を保存
        face_image.save("detected_face.jpg")
        print("detected_face.jpg に保存しました。")
    else:
        print("null")


In [28]:
def search_face_with_box():
    # コレクションIDと画像ファイル名を入力
    collection_id = input("コレクションIDを入力: ")
    image_file = input("検索する画像ファイル名を入力: ")
    # サービスクライアントを作成
    rekognition = boto3.client('rekognition')

    # 画像を読み込む
    with open(image_file, 'rb') as image:
        response = rekognition.search_faces_by_image(
            CollectionId=collection_id,
            Image={'Bytes': image.read()},
            MaxFaces=1,
            FaceMatchThreshold=80
        )

    # 結果のjsonを整形して表示
    print(json.dumps(response, indent=4))

    # 顔が見つかった場合
    if response['FaceMatches']:
        # 画像を開く
        image = Image.open(image_file)
        width, height = image.size
        # 描画オブジェクトを作成
        draw = ImageDraw.Draw(image)
        face = response['FaceMatches'][0]['Face']['BoundingBox']
        # 顔の位置情報を取得
        left = int(face['Left'] * width)
        top = int(face['Top'] * height)
        right = left + int(face['Width'] * width)
        bottom = top + int(face['Height'] * height)
         # 赤枠を描画
        draw.rectangle([left, top, right, bottom], outline="red", width=5)
        # 画像を保存
        image.save("detected_face_box.jpg")
        print("detected_face_box.jpg に保存しました。")
    else:
        print("null")

In [29]:
def delete_collection():
    # サービスクライアントを作成
    rekognition = boto3.client('rekognition')
    # 既存のコレクション一覧を取得
    response = rekognition.list_collections()
    collections = response.get('CollectionIds', [])
    print("存在するコレクション:")
    for collection in collections:
        print(f" - {collection}")
    # コレクションIDを入
    collection_id = input("削除するコレクションIDを入力: ")
    # 指定したコレクションを削除
    response = rekognition.delete_collection(CollectionId=collection_id)

    # 結果のjsonを整形して表示
    print(json.dumps(response, indent=4))
    print(f"'{collection_id}' を削除しました。")

In [22]:
def main():
    while True:
        print("1. コレクションを作成")
        print("2. 画像を登録")
        print("3. 顔を検索（顔切り取り）")
        print("4. 顔を検索（赤枠表示）")
        print("5. コレクションを削除")
        print("0. 終了")
        choice = input()
        if choice == "1":
            create_collection()
        elif choice == "2":
            add_image_to_collection()
        elif choice == "3":
            search_face()
        elif choice == "4":
            search_face_with_box()
        elif choice == "5":
            delete_collection()
        elif choice == "0":
            print("終了します。")
            break
        else:
            print("無効な選択です。")

In [30]:
if __name__ == "__main__":
    main()

1. コレクションを作成
2. 画像を登録
3. 顔を検索（顔切り取り）
4. 顔を検索（赤枠表示）
5. コレクションを削除
0. 終了


 1
コレクションIDを入力:  id1


{
    "StatusCode": 200,
    "CollectionArn": "aws:rekognition:us-east-1:502261720534:collection/id1",
    "FaceModelVersion": "7.0",
    "ResponseMetadata": {
        "RequestId": "e70c7644-ce2b-481d-b940-de397ad3f308",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "e70c7644-ce2b-481d-b940-de397ad3f308",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "115",
            "date": "Thu, 27 Feb 2025 05:06:04 GMT"
        },
        "RetryAttempts": 0
    }
}
1. コレクションを作成
2. 画像を登録
3. 顔を検索（顔切り取り）
4. 顔を検索（赤枠表示）
5. コレクションを削除
0. 終了


 2
コレクションIDを入力:  id1
登録する画像ファイル名を入力:  red.jpg


{
    "FaceRecords": [],
    "FaceModelVersion": "7.0",
    "UnindexedFaces": [],
    "ResponseMetadata": {
        "RequestId": "84885b01-217d-4e28-9333-469760470383",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "84885b01-217d-4e28-9333-469760470383",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "63",
            "date": "Thu, 27 Feb 2025 05:08:29 GMT"
        },
        "RetryAttempts": 0
    }
}
登録された顔の数: 0
1. コレクションを作成
2. 画像を登録
3. 顔を検索（顔切り取り）
4. 顔を検索（赤枠表示）
5. コレクションを削除
0. 終了


...

存在するコレクション:
 - ID1
 - id1


削除するコレクションIDを入力:  id1


{
    "StatusCode": 200,
    "ResponseMetadata": {
        "RequestId": "cc97066b-d222-4977-8878-6258b859dbb9",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "cc97066b-d222-4977-8878-6258b859dbb9",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "18",
            "date": "Thu, 27 Feb 2025 05:11:39 GMT"
        },
        "RetryAttempts": 0
    }
}
'id1' を削除しました。
1. コレクションを作成
2. 画像を登録
3. 顔を検索（顔切り取り）
4. 顔を検索（赤枠表示）
5. コレクションを削除
0. 終了


 0


終了します。
